In [ ]:
pip install dmba

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import dmba

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

from dmba import classificationSummary

from pandas.plotting import parallel_coordinates

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge

%matplotlib inline

In [ ]:
df = pd.read_excel('https://github.com/EthanRosehart/schulich_data_science/raw/main/Modified_AI-Human-PewData.xlsx')

-- For our presentation we can briefly mention again how data was filtered to only relate to the 2 topics we are investigatingreport

In [ ]:
df.columns

Index(['Unnamed: 0', 'QKEY', 'INTERVIEW_START_W99', 'INTERVIEW_END_W99',
       'DEVICE_TYPE_W99', 'LANG_W99', 'FORM_W99', 'XTABLET_W99', 'TECH1_W99',
       'SC1_W99',
       ...
       'F_PARTYSUM_FINAL', 'F_PARTYSUMIDEO_FINAL', 'F_INC_SDT1', 'F_REG',
       'F_IDEO', 'F_INTFREQ', 'F_VOLSUM', 'F_INC_TIER2', 'WEIGHT_W99',
       'political_leaning_index'],
      dtype='object', length=141)

In [ ]:
df.head()

,Unnamed: 0,QKEY,INTERVIEW_START_W99,INTERVIEW_END_W99,DEVICE_TYPE_W99,LANG_W99,FORM_W99,XTABLET_W99,TECH1_W99,SC1_W99,...,F_PARTYSUM_FINAL,F_PARTYSUMIDEO_FINAL,F_INC_SDT1,F_REG,F_IDEO,F_INTFREQ,F_VOLSUM,F_INC_TIER2,WEIGHT_W99,political_leaning_index
0,0,100260,2021-11-03 14:25:27,2021-11-03 14:45:34,Laptop/PC,English,Form 2,Non-tablet HH,NaN,Mostly positive,...,Rep/Lean Rep,Conservative Rep/Lean,"$50,000 to less than $60,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Very conservative,Several times a day,No,Middle income,0.206396,-2
1,1,100314,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,Form 1,Non-tablet HH,Mostly positive,NaN,...,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Liberal,Several times a day,Yes,Middle income,0.315090,-1
2,2,100363,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,Form 1,Non-tablet HH,Mostly positive,NaN,...,Dem/Lean Dem,Moderate/Conservative Dem/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Moderate,Several times a day,No,Upper income,0.829579,1
3,3,100598,2021-11-02 13:01:05,2021-11-04 12:37:42,Laptop/PC,English,Form 2,Non-tablet HH,NaN,Mostly positive,...,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,Several times a day,Yes,Upper income,0.337527,-2
4,4,100637,2021-11-02 12:32:58,2021-11-02 12:46:23,Laptop/PC,English,Form 2,Non-tablet HH,NaN,Equal positive and negative effects,...,Rep/Lean Rep,Conservative Rep/Lean,"$30,000 to less than $40,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Very conservative,Less often,No,Lower income,1.210606,-2


In [ ]:
# Model Goals:
# 1 - Find our target market - Location, Age, Polical, Gender, Politcal, Religion
# 2 - Find our target product market
# General AI - Baseline
# Gene Editing
# Computer Chip in Brain
# Robotic Exoskeleton

In [ ]:
df['CNCEXC_W99'].unique()

array(['Equally concerned and excited', 'More excited than concerned',
       'More concerned than excited', 'Refused'], dtype=object)

In [ ]:
ranking = {
    'Equally concerned and excited': 1,
    'More excited than concerned': 1,
    'More concerned than excited': 0,
    'Refused': np.nan
}
df['CNCEXC_W99'] = df['CNCEXC_W99'].replace(ranking)

In [ ]:
df.rename(columns={'CNCEXC_W99': 'GENERAL_3'})

,Unnamed: 0,QKEY,INTERVIEW_START_W99,INTERVIEW_END_W99,DEVICE_TYPE_W99,LANG_W99,FORM_W99,XTABLET_W99,TECH1_W99,SC1_W99,...,F_PARTYSUM_FINAL,F_PARTYSUMIDEO_FINAL,F_INC_SDT1,F_REG,F_IDEO,F_INTFREQ,F_VOLSUM,F_INC_TIER2,WEIGHT_W99,political_leaning_index
0,0,100260,2021-11-03 14:25:27,2021-11-03 14:45:34,Laptop/PC,English,Form 2,Non-tablet HH,NaN,Mostly positive,...,Rep/Lean Rep,Conservative Rep/Lean,"$50,000 to less than $60,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Very conservative,Several times a day,No,Middle income,0.206396,-2
1,1,100314,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,Form 1,Non-tablet HH,Mostly positive,NaN,...,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Liberal,Several times a day,Yes,Middle income,0.315090,-1
2,2,100363,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,Form 1,Non-tablet HH,Mostly positive,NaN,...,Dem/Lean Dem,Moderate/Conservative Dem/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Moderate,Several times a day,No,Upper income,0.829579,1
3,3,100598,2021-11-02 13:01:05,2021-11-04 12:37:42,Laptop/PC,English,Form 2,Non-tablet HH,NaN,Mostly positive,...,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,Several times a day,Yes,Upper income,0.337527,-2
4,4,100637,2021-11-02 12:32:58,2021-11-02 12:46:23,Laptop/PC,English,Form 2,Non-tablet HH,NaN,Equal positive and negative effects,...,Rep/Lean Rep,Conservative Rep/Lean,"$30,000 to less than $40,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Very conservative,Less often,No,Lower income,1.210606,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10255,10255,202101013246,2021-11-02 15:06:47,2021-11-02 15:21:18,Smartphone,English,Form 2,Non-tablet HH,NaN,Mostly positive,...,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"Less than $30,000",You are NOT registered to vote at your current address,Moderate,Almost constantly,No,Lower income,0.425901,-1
10256,10256,202101013292,2021-11-02 12:18:58,2021-11-02 12:32:49,Smartphone,English,Form 2,Non-tablet HH,NaN,Mostly positive,...,Dem/Lean Dem,Liberal Dem/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Liberal,Several times a day,Yes,Upper income,0.157638,2
10257,10257,202101013344,2021-11-04 16:30:29,2021-11-06 11:08:56,Smartphone,English,Form 1,Non-tablet HH,Mostly positive,NaN,...,Dem/Lean Dem,Liberal Dem/Lean,"Less than $30,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Liberal,Almost constantly,Yes,Lower income,0.282079,2
10258,10258,202101013446,2021-11-06 15:22:13,2021-11-06 16:10:36,Laptop/PC,English,Form 1,Non-tablet HH,Mostly positive,NaN,...,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Moderate,Several times a day,No,Middle income,0.441525,-1


In [ ]:
df['TECH1_W99'].unique()

array([nan, 'Mostly positive', 'Mostly negative',
       'Equal positive and negative effects', 'Refused'], dtype=object)

In [ ]:
ranking = {
  'Mostly positive': 1,
  'Mostly negative': 0,
  'Equal positive and negative effects': 1,
  'Refused': np.nan
}

df['TECH1_W99'] = df['TECH1_W99'].replace(ranking)

In [ ]:
df.rename(columns={'TECH1_W99': 'GENERAL_1'}, inplace=True)

In [ ]:
df['SC1_W99'].unique()

array(['Mostly positive', nan, 'Equal positive and negative effects',
       'Mostly negative', 'Refused'], dtype=object)

In [ ]:
ranking = {
  'Mostly positive': 1,
  'Mostly negative': 0,
  'Equal positive and negative effects': 1,
  'Refused': np.nan
}

df['SC1_W99'] = df['SC1_W99'].replace(ranking)

In [ ]:
df.rename(columns={'SC1_W99': 'GENERAL_2'},inplace=True)

In [ ]:
df.head(15)

,Unnamed: 0,QKEY,INTERVIEW_START_W99,INTERVIEW_END_W99,DEVICE_TYPE_W99,LANG_W99,FORM_W99,XTABLET_W99,GENERAL_1,GENERAL_2,...,F_PARTYSUM_FINAL,F_PARTYSUMIDEO_FINAL,F_INC_SDT1,F_REG,F_IDEO,F_INTFREQ,F_VOLSUM,F_INC_TIER2,WEIGHT_W99,political_leaning_index
0,0,100260,2021-11-03 14:25:27,2021-11-03 14:45:34,Laptop/PC,English,Form 2,Non-tablet HH,NaN,1.0,...,Rep/Lean Rep,Conservative Rep/Lean,"$50,000 to less than $60,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Very conservative,Several times a day,No,Middle income,0.206396,-2
1,1,100314,2021-11-04 12:35:35,2021-11-04 12:55:29,Smartphone,English,Form 1,Non-tablet HH,1.0,NaN,...,Rep/Lean Rep,Moderate/Liberal Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Liberal,Several times a day,Yes,Middle income,0.315090,-1
2,2,100363,2021-11-03 20:23:43,2021-11-03 20:36:24,Smartphone,English,Form 1,Non-tablet HH,1.0,NaN,...,Dem/Lean Dem,Moderate/Conservative Dem/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Moderate,Several times a day,No,Upper income,0.829579,1
3,3,100598,2021-11-02 13:01:05,2021-11-04 12:37:42,Laptop/PC,English,Form 2,Non-tablet HH,NaN,1.0,...,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,Several times a day,Yes,Upper income,0.337527,-2
4,4,100637,2021-11-02 12:32:58,2021-11-02 12:46:23,Laptop/PC,English,Form 2,Non-tablet HH,NaN,1.0,...,Rep/Lean Rep,Conservative Rep/Lean,"$30,000 to less than $40,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Very conservative,Less often,No,Lower income,1.210606,-2
5,5,100803,2021-11-03 19:22:44,2021-11-03 19:41:04,Laptop/PC,English,Form 1,Non-tablet HH,1.0,NaN,...,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,Several times a week,Yes,Upper income,0.302004,-2
6,6,101224,2021-11-03 18:18:21,2021-11-03 18:33:41,Smartphone,English,Form 1,Non-tablet HH,0.0,NaN,...,Rep/Lean Rep,Conservative Rep/Lean,"$40,000 to less than $50,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,About once a day,Yes,Middle income,0.287719,-2
7,7,101437,2021-11-04 11:44:54,2021-11-04 12:09:16,Laptop/PC,English,Form 2,Non-tablet HH,NaN,0.0,...,Rep/Lean Rep,Conservative Rep/Lean,"Less than $30,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,Several times a day,No,Middle income,0.397488,-2
8,8,101485,2021-11-02 19:22:24,2021-11-02 19:35:23,Smartphone,English,Form 1,Non-tablet HH,1.0,NaN,...,Rep/Lean Rep,Conservative Rep/Lean,"$100,000 or more",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,Almost constantly,Yes,Upper income,0.276541,-2
9,9,101493,2021-11-02 12:19:47,2021-11-02 13:42:56,Laptop/PC,English,Form 2,Non-tablet HH,NaN,0.0,...,Rep/Lean Rep,Conservative Rep/Lean,"$70,000 to less than $80,000",You are ABSOLUTELY CERTAIN that you are registered to vote at your current address,Conservative,Several times a day,No,Middle income,0.348764,-2


In [ ]:
df['ALGFAIR_W99'].unique()

array(['Not possible', 'Possible', 'Not sure', 'Refused'], dtype=object)

In [ ]:
ranking = {
  'Possible': 1,
  'Not possible': 0,
  'Not sure': 1,
  'Refused': np.nan
}

df['ALGFAIR_W99'] = df['ALGFAIR_W99'].replace(ranking)
df.rename(columns={'ALGFAIR_W99': 'GENERAL_4'},inplace=True)

In [ ]:
df['DISCRIM1_a_W99'].unique()

array(['Not too well', 'Very well', 'Somewhat well', 'Not sure',
       'Not at all well', 'Refused'], dtype=object)

In [ ]:
ranking = {
  'Not too well':0,
  'Very well':1,
  'Somewhat well':1,
  'Not sure':1,
  'Not at all well':0,
  'Refused':np.nan
}

df['DISCRIM1_a_W99'] = df['DISCRIM1_a_W99'].replace(ranking)
df.rename(columns={'DISCRIM1_a_W99': 'GENERAL_5'},inplace=True)

In [ ]:
ranking = {
  'Not too well':0,
  'Very well':1,
  'Somewhat well':1,
  'Not sure':1,
  'Not at all well':0,
  'Refused':np.nan
}

df['DISCRIM1_b_W99'] = df['DISCRIM1_b_W99'].replace(ranking)
df.rename(columns={'DISCRIM1_b_W99': 'GENERAL_6'},inplace=True)

In [ ]:
ranking = {
  'Not too well':0,
  'Very well':1,
  'Somewhat well':1,
  'Not sure':1,
  'Not at all well':0,
  'Refused':np.nan
}

df['DISCRIM1_c_W99'] = df['DISCRIM1_c_W99'].replace(ranking)
df.rename(columns={'DISCRIM1_c_W99': 'GENERAL_7'},inplace=True)

In [ ]:
ranking = {
  'Not too well':0,
  'Very well':1,
  'Somewhat well':1,
  'Not sure':1,
  'Not at all well':0,
  'Refused':np.nan
}

df['DISCRIM1_d_W99'] = df['DISCRIM1_d_W99'].replace(ranking)
df.rename(columns={'DISCRIM1_d_W99': 'GENERAL_8'},inplace=True)

In [ ]:
ranking = {
  'Not too well':0,
  'Very well':1,
  'Somewhat well':1,
  'Not sure':1,
  'Not at all well':0,
  'Refused':np.nan
}

df['DISCRIM1_e_W99'] = df['DISCRIM1_e_W99'].replace(ranking)
df.rename(columns={'DISCRIM1_e_W99': 'GENERAL_9'},inplace=True)

In [ ]:
ranking = {
  'Not too well':0,
  'Very well':1,
  'Somewhat well':1,
  'Not sure':1,
  'Not at all well':0,
  'Refused':np.nan
}

df['DISCRIM1_f_W99'] = df['DISCRIM1_f_W99'].replace(ranking)
df.rename(columns={'DISCRIM1_f_W99': 'GENERAL_9f'},inplace=True)

In [ ]:
df['POSNEGAI_a_W99'].unique()

array(['Very concerned', 'Equal excitement and concern',
       'Somewhat concerned', 'Somewhat excited', 'Very excited',
       'Refused'], dtype=object)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGAI_a_W99'] = df['POSNEGAI_a_W99'].replace(ranking)
df.rename(columns={'POSNEGAI_a_W99': 'GENERAL_10'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGAI_b_W99'] = df['POSNEGAI_b_W99'].replace(ranking)
df.rename(columns={'POSNEGAI_b_W99': 'GENERAL_11'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGAI_c_W99'] = df['POSNEGAI_c_W99'].replace(ranking)
df.rename(columns={'POSNEGAI_c_W99': 'GENERAL_12'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGAI_d_W99'] = df['POSNEGAI_d_W99'].replace(ranking)
df.rename(columns={'POSNEGAI_d_W99': 'GENERAL_13'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGAI_e_W99'] = df['POSNEGAI_e_W99'].replace(ranking)
df.rename(columns={'POSNEGAI_e_W99': 'GENERAL_14'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGAI_f_W99'] = df['POSNEGAI_f_W99'].replace(ranking)
df.rename(columns={'POSNEGAI_f_W99': 'GENERAL_15'},inplace=True)

In [ ]:
df['POSNEGHE_a_W99'].unique()

array(['Somewhat excited', 'Equal excitement and concern', 'Refused',
       'Very concerned', 'Somewhat concerned', 'Very excited'],
      dtype=object)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGHE_a_W99'] = df['POSNEGHE_a_W99'].replace(ranking)
df.rename(columns={'POSNEGHE_a_W99': 'GENERAL_16'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGHE_b_W99'] = df['POSNEGHE_b_W99'].replace(ranking)
df.rename(columns={'POSNEGHE_b_W99': 'GENERAL_17'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGHE_c_W99'] = df['POSNEGHE_c_W99'].replace(ranking)
df.rename(columns={'POSNEGHE_c_W99': 'GENERAL_18'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGHE_d_W99'] = df['POSNEGHE_d_W99'].replace(ranking)
df.rename(columns={'POSNEGHE_d_W99': 'GENERAL_19'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGHE_e_W99'] = df['POSNEGHE_e_W99'].replace(ranking)
df.rename(columns={'POSNEGHE_e_W99': 'GENERAL_20'},inplace=True)

In [ ]:
ranking = {
  'Very concerned':0,
  'Equal excitement and concern':1,
  'Somewhat concerned':0,
  'Somewhat excited':1,
  'Very excited':1,
  'Refused': np.nan
}

df['POSNEGHE_f_W99'] = df['POSNEGHE_f_W99'].replace(ranking)
df.rename(columns={'POSNEGHE_f_W99': 'GENERAL_21'},inplace=True)

In [ ]:
df['BCHIP1_W99'].unique()

array(['Nothing at all', nan, 'A little', 'A lot', 'Refused'],
      dtype=object)

In [ ]:
ranking = {
  'Nothing at all':0,
  'A little':1,
  'A lot':1,
  'Refused': np.nan
}

df['BCHIP1_W99'] = df['BCHIP1_W99'].replace(ranking)
df.rename(columns={'BCHIP1_W99': 'CHIP_1'},inplace=True)

In [ ]:
df['BCHIP2_W99'].unique()

array(['Not sure', nan, 'Bad idea for society', 'Good idea for society',
       'Refused'], dtype=object)

In [ ]:
ranking = {
  'Not sure':1,
  'Bad idea for society':0,
  'Good idea for society':1,
}

df['BCHIP2_W99'] = df['BCHIP2_W99'].replace(ranking)
df.rename(columns={'BCHIP2_W99': 'CHIP_2'},inplace=True)

In [ ]:
df['BCHIP3_W99'].unique()

array(['Probably NOT want', nan, 'Definitely NOT want', 'Probably want',
       'Definitely want', 'Refused'], dtype=object)

In [ ]:
ranking = {
  'Probably NOT want':0,
  'Definitely NOT want':0,
  'Probably want':1,
  'Definitely want':1,
  'Refused':np.nan
}

df['BCHIP3_W99'] = df['BCHIP3_W99'].replace(ranking)
df.rename(columns={'BCHIP3_W99': 'CHIP_3'},inplace=True)

In [ ]:
# Columns to check
columns_to_check = [
    'BCHIP11_W99', 'BCHIP12_W99', 'BCHIP13_a_W99', 'BCHIP13_b_W99', 'BCHIP14_a_W99',
    'BCHIP14_b_W99', 'BCHIP14_c_W99', 'BCHIP14_d_W99', 'GENEV1_W99', 'GENEV2_W99',
    'GENEV3_W99', 'GENEV4_a_W99', 'GENEV4_b_W99', 'GENEV4_c_W99', 'GENEV4_d_W99',
    'GENEV5_W99', 'GENEV6_W99', 'GENEV7_W99', 'GENEV8_W99', 'GENEV9_a_W99',
    'GENEV9_b_W99', 'GENEV9_c_W99', 'GENEV9_d_W99', 'GENEV10_W99', 'GENEV11_W99',
    'GENEV12_a_W99', 'GENEV12_b_W99', 'GENEV12_c_W99', 'GENEV13_a_W99', 'GENEV13_b_W99',
    'EXOV1_W99', 'EXOV2_W99', 'EXOV3_W99', 'EXOV4_a_W99', 'EXOV4_b_W99',
    'EXOV4_c_W99', 'EXOV4_d_W99', 'EXOV5_W99', 'EXOV6_W99', 'EXOV7_W99',
    'EXOV8_a_W99', 'EXOV8_b_W99', 'EXOV8_c_W99', 'EXOV8_d_W99', 'EXOV9_W99',
    'EXOV10_W99', 'EXOV11_a_W99', 'EXOV11_b_W99', 'EXOV12_a_W99', 'EXOV12_b_W99',
    'EXOV12_c_W99'
]

# Get unique values for each specified column
unique_values_dict = {}
for col in columns_to_check:
    unique_values_dict[col] = df[col].unique()

# Print unique values for verification
for col, unique_values in unique_values_dict.items():
    print(f"Unique values for {col}: {unique_values}")

Unique values for BCHIP11_W99: ['Government will go too far regulating their use' nan
 'Government will not go far enough regulating their use' 'Refused']
Unique values for BCHIP12_W99: ['A higher standard than used for medical devices' nan
 'Existing standards used for medical devices' 'Refused']
Unique values for BCHIP13_a_W99: ['More acceptable' nan 'Less acceptable' 'No difference' 'Refused']
Unique values for BCHIP13_b_W99: ['More acceptable' nan 'Less acceptable' 'No difference' 'Refused']
Unique values for BCHIP14_a_W99: ['Favor' nan 'Oppose' 'Not sure' 'Refused']
Unique values for BCHIP14_b_W99: ['Favor' nan 'Not sure' 'Oppose' 'Refused']
Unique values for BCHIP14_c_W99: ['Oppose' nan 'Favor' 'Not sure' 'Refused']
Unique values for BCHIP14_d_W99: ['Not sure' nan 'Favor' 'Oppose' 'Refused']
Unique values for GENEV1_W99: ['Nothing at all' nan 'A little' 'A lot' 'Refused']
Unique values for GENEV2_W99: ['Not sure' nan 'Bad idea for society' 'Good idea for society' 'Refused']
Uniqu

In [ ]:
# Define a function to process each column
def process_column(df, col_name, ranking, new_col_name):
    df[col_name] = df[col_name].replace(ranking)
    df.rename(columns={col_name: new_col_name}, inplace=True)
    return df

# Manually create the ranking dictionaries based on unique values

# BCHIP4_a_W99
ranking_BCHIP4_a_W99 = {
  'Probably would NOT happen': 0,
  'Definitely would NOT happen': 0,
  'Definitely would happen': 1,
  'Probably would happen': 1,
  'Refused': np.nan
}
df = process_column(df, 'BCHIP4_a_W99', ranking_BCHIP4_a_W99, 'CHIP_4_a')

# BCHIP4_b_W99
ranking_BCHIP4_b_W99 = {
  'Probably would NOT happen': 0,
  'Definitely would NOT happen': 0,
  'Definitely would happen': 1,
  'Probably would happen': 1,
  'Refused': np.nan
}
df = process_column(df, 'BCHIP4_b_W99', ranking_BCHIP4_b_W99, 'CHIP_4_b')

# BCHIP4_c_W99
ranking_BCHIP4_c_W99 = {
  'Probably would NOT happen': 0,
  'Definitely would NOT happen': 0,
  'Definitely would happen': 1,
  'Probably would happen': 1,
  'Refused': np.nan
}
df = process_column(df, 'BCHIP4_c_W99', ranking_BCHIP4_c_W99, 'CHIP_4_c')

# BCHIP4_d_W99
ranking_BCHIP4_d_W99 = {
  'Probably would NOT happen': 0,
  'Definitely would NOT happen': 0,
  'Definitely would happen': 1,
  'Probably would happen': 1,
  'Refused': np.nan
}
df = process_column(df, 'BCHIP4_d_W99', ranking_BCHIP4_d_W99, 'CHIP_4_d')

# BCHIP5_W99
ranking_BCHIP5_W99 = {
  'This idea is meddling with nature and crosses a line we should not cross': 0,

  'As humans, we are always trying to better ourselves and this idea is no different': 1,
  'Refused': np.nan
}
df = process_column(df, 'BCHIP5_W99', ranking_BCHIP5_W99, 'CHIP_5')

# BCHIP6_W99
ranking_BCHIP6_W99 = {
  'Feel pressure to get this': 0,
  'Not feel pressure to get this': 1,
  'Refused': np.nan
}
df = process_column(df, 'BCHIP6_W99', ranking_BCHIP6_W99, 'CHIP_6')

# BCHIP7_W99
ranking_BCHIP7_W99 = {
  'Worse than now': 0,
  'About the same as now': 1,
  'Better than now': 1,
  'Refused': np.nan
}
df = process_column(df, 'BCHIP7_W99', ranking_BCHIP7_W99, 'CHIP_7')

# BCHIP8_W99
ranking_BCHIP8_W99 = {
    'Increase the gap between higher and lower-income Americans': 1,
    'Decrease the gap between higher and lower-income Americans': 0,
    'Not make much difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP8_W99', ranking_BCHIP8_W99, 'CHIP_8')

# BCHIP9_a_W99
ranking_BCHIP9_a_W99 = {
    'Rarely or never': 0,
    'Some of the time': 1,
    'A lot of the time': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP9_a_W99', ranking_BCHIP9_a_W99, 'CHIP_9_a')

# BCHIP9_b_W99
ranking_BCHIP9_b_W99 = {
    'Rarely or never': 0,
    'Some of the time': 1,
    'A lot of the time': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP9_b_W99', ranking_BCHIP9_b_W99, 'CHIP_9_b')

# BCHIP9_c_W99
ranking_BCHIP9_c_W99 = {
    'Rarely or never': 0,
    'Some of the time': 1,
    'A lot of the time': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP9_c_W99', ranking_BCHIP9_c_W99, 'CHIP_9_c')

# BCHIP10_a_W99
ranking_BCHIP10_a_W99 = {
    'A major role': 1,
    'No role at all': 0,
    'A minor role': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP10_a_W99', ranking_BCHIP10_a_W99, 'CHIP_10_a')

# BCHIP10_b_W99
ranking_BCHIP10_b_W99 = {
    'A major role': 1,
    'No role at all': 0,
    'A minor role': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP10_b_W99', ranking_BCHIP10_b_W99, 'CHIP_10_b')

# BCHIP10_c_W99
ranking_BCHIP10_c_W99 = {
    'A major role': 1,
    'No role at all': 0,
    'A minor role': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP10_c_W99', ranking_BCHIP10_c_W99, 'CHIP_10_c')

# BCHIP10_d_W99
ranking_BCHIP10_d_W99 = {
    'A major role': 1,
    'No role at all': 0,
    'A minor role': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP10_d_W99', ranking_BCHIP10_d_W99, 'CHIP_10_d')

# BCHIP11_W99
ranking_BCHIP11_W99 = {
    'Government will go too far regulating their use': 1,
    'Government will not go far enough regulating their use': 0,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP11_W99', ranking_BCHIP11_W99, 'CHIP_11')

# BCHIP12_W99
ranking_BCHIP12_W99 = {
    'A higher standard than used for medical devices': 0,
    'Existing standards used for medical devices': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP12_W99', ranking_BCHIP12_W99, 'CHIP_12')

# BCHIP13_a_W99
ranking_BCHIP13_a_W99 = {
    'More acceptable': 1,
    'Less acceptable': 0,
    'No difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP13_a_W99', ranking_BCHIP13_a_W99, 'CHIP_13_a')

# BCHIP13_b_W99
ranking_BCHIP13_b_W99 = {
    'More acceptable': 1,
    'Less acceptable': 0,
    'No difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP13_b_W99', ranking_BCHIP13_b_W99, 'CHIP_13_b')

# BCHIP14_a_W99
ranking_BCHIP14_a_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP14_a_W99', ranking_BCHIP14_a_W99, 'CHIP_14_a')

# BCHIP14_b_W99
ranking_BCHIP14_b_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP14_b_W99', ranking_BCHIP14_b_W99, 'CHIP_14_b')

# BCHIP14_c_W99
ranking_BCHIP14_c_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP14_c_W99', ranking_BCHIP14_c_W99, 'CHIP_14_c')

# BCHIP14_d_W99
ranking_BCHIP14_d_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'BCHIP14_d_W99', ranking_BCHIP14_d_W99, 'CHIP_14_d')

# GENEV1_W99
ranking_GENEV1_W99 = {
    'A lot': 1,
    'A little': 1,
    'Nothing at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV1_W99', ranking_GENEV1_W99, 'GENEV_1')

# GENEV2_W99
ranking_GENEV2_W99 = {
    'Good idea for society': 1,
    'Bad idea for society': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV2_W99', ranking_GENEV2_W99, 'GENEV_2')

# GENEV3_W99
ranking_GENEV3_W99 = {
    'Yes, I would definitely want this for my baby': 1,
    'No, I would definitely NOT want this for my baby': 0,
    'Yes, I would probably want this for my baby': 1,
    'No, I would probably NOT want this for my baby': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV3_W99', ranking_GENEV3_W99, 'GENEV_3')

# GENEV4_a_W99
ranking_GENEV4_a_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV4_a_W99', ranking_GENEV4_a_W99, 'GENEV_4_a')

# GENEV4_b_W99
ranking_GENEV4_b_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV4_b_W99', ranking_GENEV4_b_W99, 'GENEV_4_b')

# GENEV4_c_W99
ranking_GENEV4_c_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV4_c_W99', ranking_GENEV4_c_W99, 'GENEV_4_c')

# GENEV4_d_W99
ranking_GENEV4_d_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV4_d_W99', ranking_GENEV4_d_W99, 'GENEV_4_d')

# GENEV5_W99
ranking_GENEV5_W99 = {
    'As humans, we are always trying to better ourselves and this idea is no different': 1,
    'This idea is meddling with nature and crosses a line we should not cross': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV5_W99', ranking_GENEV5_W99, 'GENEV_5')

# GENEV6_W99
ranking_GENEV6_W99 = {
    'Not feel pressure to get this for their baby': 1,
    'Feel pressure to get this for their baby': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV6_W99', ranking_GENEV6_W99, 'GENEV_6')

# GENEV7_W99
ranking_GENEV7_W99 = {
    'Better than now': 1,
    'About the same as now': 1,
    'Worse than now': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV7_W99', ranking_GENEV7_W99, 'GENEV_7')

# GENEV8_W99
ranking_GENEV8_W99 = {
    'Increase the gap between higher and lower-income Americans': 0,
    'Decrease the gap between higher and lower-income Americans': 1,
    'Not make much difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV8_W99', ranking_GENEV8_W99, 'GENEV_8')

# GENEV9_a_W99
ranking_GENEV9_a_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV9_a_W99', ranking_GENEV9_a_W99, 'GENEV_9_a')

# GENEV9_b_W99
ranking_GENEV9_b_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV9_b_W99', ranking_GENEV9_b_W99, 'GENEV_9_b')

# GENEV9_c_W99
ranking_GENEV9_c_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV9_c_W99', ranking_GENEV9_c_W99, 'GENEV_9_c')

# GENEV9_d_W99
ranking_GENEV9_d_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV9_d_W99', ranking_GENEV9_d_W99, 'GENEV_9_d')

# GENEV10_W99
ranking_GENEV10_W99 = {
    'Government will go too far regulating their use': 1,
    'Government will not go far enough regulating their use': 0,
    'Refused': np.nan
}
df = process_column(df, 'GENEV10_W99', ranking_GENEV10_W99, 'GENEV_10')

# GENEV11_W99
ranking_GENEV11_W99 = {
    'A higher standard than used for medical treatments': 0,
    'Existing standards used for medical treatments': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV11_W99', ranking_GENEV11_W99, 'GENEV_11')

# GENEV12_a_W99
ranking_GENEV12_a_W99 = {
    'More acceptable': 1,
    'Less acceptable': 0,
    'No difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV12_a_W99', ranking_GENEV12_a_W99, 'GENEV_12_a')

# GENEV12_b_W99
ranking_GENEV12_b_W99 = {
    'More acceptable': 1,
    'Less acceptable': 0,
    'No difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV12_b_W99', ranking_GENEV12_b_W99, 'GENEV_12_b')

# GENEV12_c_W99
ranking_GENEV12_c_W99 = {
    'More acceptable': 1,
    'Less acceptable': 0,
    'No difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV12_c_W99', ranking_GENEV12_c_W99, 'GENEV_12_c')

# GENEV13_a_W99
ranking_GENEV13_a_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV13_a_W99', ranking_GENEV13_a_W99, 'GENEV_13_a')

# GENEV13_b_W99
ranking_GENEV13_b_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'GENEV13_b_W99', ranking_GENEV13_b_W99, 'GENEV_13_b')

# EXOV1_W99
ranking_EXOV1_W99 = {
    'A lot': 1,
    'A little': 1,
    'Nothing at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV1_W99', ranking_EXOV1_W99, 'EXOV_1')

# EXOV2_W99
ranking_EXOV2_W99 = {
    'Good idea for society': 1,
    'Bad idea for society': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV2_W99', ranking_EXOV2_W99, 'EXOV_2')

# EXOV3_W99
ranking_EXOV3_W99 = {
    'Definitely want': 1,
    'Probably want': 1,
    'Definitely NOT want': 0,
    'Probably NOT want': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV3_W99', ranking_EXOV3_W99, 'EXOV_3')

# EXOV4_a_W99
ranking_EXOV4_a_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV4_a_W99', ranking_EXOV4_a_W99, 'EXOV_4_a')

# EXOV4_b_W99
ranking_EXOV4_b_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV4_b_W99', ranking_EXOV4_b_W99, 'EXOV_4_b')

# EXOV4_c_W99
ranking_EXOV4_c_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV4_c_W99', ranking_EXOV4_c_W99, 'EXOV_4_c')

# EXOV4_d_W99
ranking_EXOV4_d_W99 = {
    'Definitely would happen': 1,
    'Probably would happen': 1,
    'Probably would NOT happen': 0,
    'Definitely would NOT happen': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV4_d_W99', ranking_EXOV4_d_W99, 'EXOV_4_d')

# EXOV5_W99
ranking_EXOV5_W99 = {
    'As humans, we are always trying to better ourselves and this idea is no different': 1,
    'This idea is meddling with nature and crosses a line we should not cross': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV5_W99', ranking_EXOV5_W99, 'EXOV_5')

# EXOV6_W99
ranking_EXOV6_W99 = {
    'Better than now': 1,
    'About the same as now': 1,
    'Worse than now': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV6_W99', ranking_EXOV6_W99, 'EXOV_6')

# EXOV7_W99
ranking_EXOV7_W99 = {
    'Robotic exoskeletons should only be made if they fit a wide range of worker body types, even if that increases their cost': 1,
    'It’s okay to make robotic exoskeletons that just fit the typical body types of manual labor workers in order to lower the cost': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV7_W99', ranking_EXOV7_W99, 'EXOV_7')

# EXOV8_a_W99
ranking_EXOV8_a_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV8_a_W99', ranking_EXOV8_a_W99, 'EXOV_8_a')

# EXOV8_b_W99
ranking_EXOV8_b_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV8_b_W99', ranking_EXOV8_b_W99, 'EXOV_8_b')

# EXOV8_c_W99
ranking_EXOV8_c_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV8_c_W99', ranking_EXOV8_c_W99, 'EXOV_8_c')

# EXOV8_d_W99
ranking_EXOV8_d_W99 = {
    'A major role': 1,
    'A minor role': 1,
    'No role at all': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV8_d_W99', ranking_EXOV8_d_W99, 'EXOV_8_d')

# EXOV9_W99
ranking_EXOV9_W99 = {
    'Government will go too far regulating their use': 1,
    'Government will not go far enough regulating their use': 0,
    'Refused': np.nan
}
df = process_column(df, 'EXOV9_W99', ranking_EXOV9_W99, 'EXOV_9')

# EXOV10_W99
ranking_EXOV10_W99 = {
    'A higher standard than used for workplace equipment': 0,
    'Existing standards used for workplace equipment': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV10_W99', ranking_EXOV10_W99, 'EXOV_10')

# EXOV11_a_W99
ranking_EXOV11_a_W99 = {
    'More acceptable': 1,
    'Less acceptable': 0,
    'No difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV11_a_W99', ranking_EXOV11_a_W99, 'EXOV_11_a')

# EXOV11_b_W99
ranking_EXOV11_b_W99 = {
    'More acceptable': 1,
    'Less acceptable': 0,
    'No difference': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV11_b_W99', ranking_EXOV11_b_W99, 'EXOV_11_b')

# EXOV12_a_W99
ranking_EXOV12_a_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV12_a_W99', ranking_EXOV12_a_W99, 'EXOV_12_a')

# EXOV12_b_W99
ranking_EXOV12_b_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV12_b_W99', ranking_EXOV12_b_W99, 'EXOV_12_b')

# EXOV12_c_W99
ranking_EXOV12_c_W99 = {
    'Favor': 1,
    'Oppose': 0,
    'Not sure': 1,
    'Refused': np.nan
}
df = process_column(df, 'EXOV12_c_W99', ranking_EXOV12_c_W99, 'EXOV_12_c')

In [ ]:
columns_to_drop = [
    'EXCITEOE_W99_OE1', 'EXCITEOE_W99_OE2', 'EXCITEOE_W99_OE3', 'EXCITEOE_REFUSED_W99',
    'CONCERNOE_W99_OE1', 'CONCERNOE_W99_OE2', 'CONCERNOE_W99_OE3', 'CONCERNOE_REFUSED_W99'
]

# Drop the columns
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
df.drop(columns='EXOV_7', inplace=True)

In [ ]:
df.to_csv('transform3.csv', index=False)